In [63]:
from __future__ import annotations
from dataclasses import dataclass
from typing import List, Optional, Dict, Any, Iterable
import hashlib
import tiktoken
from langchain_text_splitters import RecursiveCharacterTextSplitter
from FlagEmbedding import BGEM3FlagModel, FlagReranker
import numpy as np
import math
from typing import Sequence, Iterable


# Dataclasses

In [8]:
@dataclass
class Chunk:
    id: str
    text: str
    metadata: Dict[str, Any]
    

# Utilities

In [43]:
def _stable_chunk_id(source_id: str, chunk_index: int, text: str) -> str:
    h = hashlib.sha256(text.encode("utf-8")).hexdigest()[:16]
    return f"{source_id}:{chunk_index}:{h}"

def load_documents() -> List[Dict[str, Any]]:
    return [
        {
            "source_id": "policy_returns_tr",
            "title": "İade Politikası",
            "url": "https://example.com/iade",
            "text": (
                "İade politikamız aşağıdaki gibidir.\n\n"
                "1) Teslimat tarihinden itibaren 14 gün içinde iade talebi oluşturabilirsiniz. "
                "Ürün kullanılmamış olmalıdır.\n\n"
                "2) Kargo takip numarası sipariş sayfasında yer alır. "
                "İade kargo bedeli kampanya dönemlerinde ücretsiz olabilir.\n\n"
                "3) İade onayı sonrasında ücret iadesi 3-10 iş günü içinde gerçekleştirilir.\n"
            ),
        },
        {
            "source_id": "policy_privacy_tr",
            "title": "Gizlilik Politikası",
            "url": "https://example.com/gizlilik",
            "text": (
                "Gizlilik politikamız aşağıdaki gibidir.\n\n"
                "1) Kişisel verileriniz, hizmet sunumu ve müşteri desteği amaçlarıyla işlenir.\n\n"
                "2) Verileriniz, yasal yükümlülükler veya açık rızanız olmaksızın üçüncü taraflarla paylaşılmaz.\n\n"
                "3) Çerezler (cookies), deneyimi iyileştirmek ve analiz yapmak için kullanılabilir. "
                "Tarayıcı ayarlarınızdan çerezleri yönetebilirsiniz.\n\n"
                "4) Veri saklama süresi, ilgili mevzuata uygun şekilde belirlenir; gerekli olmadığında veriler silinir veya anonimleştirilir.\n"
            )
        }
    ]

def print_chunk_stats(chunks: List[Chunk], top_n: int = 5) -> None:
    if not chunks:
        print("No chunks.")
        return

    token_counts = [c.metadata["token_count"] for c in chunks]
    print(f"chunks: {len(chunks)}")
    print(f"min/avg/max tokens: {min(token_counts)} / {sum(token_counts)/len(token_counts):.1f} / {max(token_counts)}")

    # En uzun birkaç chunk'a bak (önizleme)
    longest = sorted(chunks, key=lambda c: c.metadata["token_count"], reverse=True)[:top_n]
    print("\nLongest chunks:")
    for c in longest:
        preview = c.text[:160].replace("\n", " ")
        print(f"- {c.id} | {c.metadata['token_count']} tok | {preview}...")

def sigmoid(x):
    x = np.asarray(x)
    return 1.0 / (1.0 + np.exp(-x))

# Chunking

In [10]:
def build_text_splitter(
        *,
        encoding_name: str = "o200k_base",
        chunk_size: int = 600,   # token cinsinden hedef
        chunk_overlap: int = 80, # token cinsinden overlap
) -> tuple[RecursiveCharacterTextSplitter, Any]:
    enc = tiktoken.get_encoding(encoding_name)

    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        encoding_name=encoding_name,
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=[
            "\n\n",         # paragraf
            "\n",           # satır
            ". ",           # cümle
            "? ",
            "! ",
            "; ",
            ": ",
            ", ",
            " ",
            ""              # last resort: karakter bazlı
        ],
        add_start_index=True,
    )

    return splitter, enc


def chunk_documents(
        docs: Iterable[Dict[str, Any]],
        *,
        encoding_name: str = "o200k_base",
        chunk_size: int = 600,
        chunk_overlap: int = 80,
        text_key: str = "text",
        source_id_key: str = "source_id",
) -> List[Chunk]:
    """
    docs input örneği:
      {"source_id": "policy_001", "text": "...", "title": "...", "url": "..."}
    """
    splitter, enc = build_text_splitter(
        encoding_name=encoding_name,
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )

    out: List[Chunk] = []

    for doc in docs:
        source_id = str(doc.get(source_id_key) or doc.get("id") or "unknown")
        text = str(doc.get(text_key) or "")

        pieces = splitter.create_documents([text], metadatas=[{k: v for k, v in doc.items() if k != text_key}])
        for i, p in enumerate(pieces):
            chunk_text = p.page_content.strip()
            if not chunk_text:
                continue

            token_count = len(enc.encode(chunk_text))
            md = dict(p.metadata or {})
            md.update({
                "source_id": source_id,
                "chunk_index": i,
                "token_count": token_count,
            })

            chunk_id = _stable_chunk_id(source_id, i, chunk_text)
            out.append(Chunk(id=chunk_id, text=chunk_text, metadata=md))
    return out

In [28]:
sample_docs = load_documents()
chunks = chunk_documents(
    sample_docs,
    encoding_name="o200k_base", # 200k için optimize edilmiş tokenizer
    chunk_size=30,       # demo için küçük
    chunk_overlap=3,
)

print_chunk_stats(chunks)

chunks: 11
min/avg/max tokens: 9 / 19.8 / 29

Longest chunks:
- policy_returns_tr:2:4071d5867bcd5a10 | 29 tok | 2) Kargo takip numarası sipariş sayfasında yer alır. İade kargo bedeli kampanya dönemlerinde ücretsiz olabilir....
- policy_privacy_tr:2:00de60f5b8c61801 | 29 tok | 2) Verileriniz, yasal yükümlülükler veya açık rızanız olmaksızın üçüncü taraflarla paylaşılmaz....
- policy_returns_tr:1:e7c0352d38efe0e9 | 28 tok | 1) Teslimat tarihinden itibaren 14 gün içinde iade talebi oluşturabilirsiniz. Ürün kullanılmamış olmalıdır....
- policy_privacy_tr:1:455df706c869ad96 | 24 tok | 1) Kişisel verileriniz, hizmet sunumu ve müşteri desteği amaçlarıyla işlenir....
- policy_returns_tr:3:86e05d8b3137f8c4 | 23 tok | 3) İade onayı sonrasında ücret iadesi 3-10 iş günü içinde gerçekleştirilir....


# Embedding

In [29]:
model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=True) # GPU varsa fp16 kullan
docs = [c.text for c in chunks]
doc_out = model.encode(
    docs,
    batch_size=32,
    max_length=1024,
    return_dense=True,
    return_sparse=False,
    return_colbert_vecs=False
)

doc_vecs = doc_out["dense_vecs"] # (N, 1024)
for i in range(len(docs)):
    print(f"Doc {i} vec:", doc_vecs[i], len(doc_vecs[i]))

Doc 0 vec: [-0.0392    0.002842 -0.03415  ... -0.01094   0.00143  -0.03078 ] 1024
Doc 1 vec: [-0.0182   0.03668 -0.02942 ... -0.05847 -0.0242  -0.03143] 1024
Doc 2 vec: [-0.06033 -0.0399  -0.0442  ... -0.0414  -0.0468  -0.02924] 1024
Doc 3 vec: [-0.00823   0.012146 -0.0435   ... -0.02716  -0.03494   0.00132 ] 1024
Doc 4 vec: [-0.05148 -0.01796 -0.02898 ...  0.01878  0.01582 -0.03503] 1024
Doc 5 vec: [-0.03537  -0.01558  -0.07996  ... -0.003641 -0.00971  -0.0587  ] 1024
Doc 6 vec: [-0.0503    0.01944  -0.002491 ...  0.02812  -0.02069  -0.02695 ] 1024
Doc 7 vec: [-0.01929  0.01148 -0.03467 ...  0.01822 -0.00712 -0.04102] 1024
Doc 8 vec: [-0.0298     0.00227   -0.01768   ...  0.003473  -0.04248   -0.0001634] 1024
Doc 9 vec: [-0.02423  0.00647 -0.03177 ...  0.00626 -0.01044 -0.03384] 1024
Doc 10 vec: [-0.03278   0.00945  -0.04404  ...  0.03668   0.005215 -0.04715 ] 1024


# Similarities

In [13]:
def dot(a: np.ndarray, b: np.ndarray) -> float:
    return float(np.dot(a, b))


def l2_norm(a: np.ndarray) -> float:
    return float(np.linalg.norm(a))


def cosine_similarity(a: np.ndarray, b: np.ndarray, eps: float = 1e-12) -> float:
    denom = max(l2_norm(a) * l2_norm(b), eps)
    return dot(a, b) / denom


def euclidean_distance(a: np.ndarray, b: np.ndarray) -> float:
    return float(np.linalg.norm(a - b))


def l2_normalize(a: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    n = np.linalg.norm(a)
    return a / max(n, eps)

# Similarity Search Hands on

## Cosine Similarity [-1,1] aralığında, 1'e ne kadar yakın o kadar benzer

## Euclidean Distance [0, ∞) aralığında, 0'a ne kadar yakın o kadar benzer

## Dot Product [−∞, ∞) aralığında, büyük değerler daha benzer

# Yöntem

- Soruyu embedding ile vektöre çevir
- Vector databaseindeki tüm vectorler ile Cosine, Euclidean ve Dot Product karşılaştır
- Yukarıdaki şartlarda benzerlikleri bul.

## Training: Farklı chunk sizelar ile aynı işlemleri dene

In [57]:
question = "Ücret iade süreci nedir?"

In [58]:

question_vector = model.encode(
    [question],
    batch_size=1,
    max_length=1024,
    return_dense=True,
    return_sparse=False,
    return_colbert_vecs=False,
)["dense_vecs"][0]  # (1024,)

for i in range(len(docs)):
    cosine = cosine_similarity(question_vector, doc_vecs[i])
    euclidean = euclidean_distance(question_vector, doc_vecs[i])
    dot_product = dot(question_vector, doc_vecs[i])

    print(f"Doc {i} | {docs[i][:30]} | Cosine sim: {cosine:.4f} | Euclidean dist: {euclidean:.4f} | Dot prod: {dot_product:.4f}")

Inference Embeddings: 100%|██████████| 1/1 [00:00<00:00, 26.93it/s]

Doc 0 | İade politikamız aşağıdaki gib | Cosine sim: 0.5921 | Euclidean dist: 0.9033 | Dot prod: 0.5918
Doc 1 | 1) Teslimat tarihinden itibare | Cosine sim: 0.6240 | Euclidean dist: 0.8672 | Dot prod: 0.6240
Doc 2 | 2) Kargo takip numarası sipari | Cosine sim: 0.5271 | Euclidean dist: 0.9727 | Dot prod: 0.5269
Doc 3 | 3) İade onayı sonrasında ücret | Cosine sim: 0.6572 | Euclidean dist: 0.8286 | Dot prod: 0.6572
Doc 4 | Gizlilik politikamız aşağıdaki | Cosine sim: 0.3630 | Euclidean dist: 1.1289 | Dot prod: 0.3630
Doc 5 | 1) Kişisel verileriniz, hizmet | Cosine sim: 0.4028 | Euclidean dist: 1.0938 | Dot prod: 0.4028
Doc 6 | 2) Verileriniz, yasal yükümlül | Cosine sim: 0.3477 | Euclidean dist: 1.1426 | Dot prod: 0.3477
Doc 7 | 3) Çerezler (cookies), deneyim | Cosine sim: 0.3406 | Euclidean dist: 1.1484 | Dot prod: 0.3406
Doc 8 | . Tarayıcı ayarlarınızdan çere | Cosine sim: 0.3131 | Euclidean dist: 1.1719 | Dot prod: 0.3130
Doc 9 | 4) Veri saklama süresi, ilgili | Cosine sim: 0.3962 | Eu

# Reranking

In [59]:
reranker_model = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

candidates = [d for d in docs]
similiarities = []

print("Before reranking:")
for i, c in enumerate(candidates):
    cosine = cosine_similarity(question_vector, doc_vecs[i])
    #print(f"Candidate {i} | Cosine sim: {cosine:.4f} | {c[:50]}...")
    similiarities.append((i, cosine))


similiarities.sort(key=lambda x: x[1], reverse=True)
for i, score in similiarities:
    print(f"(Score: {score:.4f}) {candidates[i]}") 


Before reranking:
(Score: 0.6572) 3) İade onayı sonrasında ücret iadesi 3-10 iş günü içinde gerçekleştirilir.
(Score: 0.6240) 1) Teslimat tarihinden itibaren 14 gün içinde iade talebi oluşturabilirsiniz. Ürün kullanılmamış olmalıdır.
(Score: 0.5921) İade politikamız aşağıdaki gibidir.
(Score: 0.5271) 2) Kargo takip numarası sipariş sayfasında yer alır. İade kargo bedeli kampanya dönemlerinde ücretsiz olabilir.
(Score: 0.4028) 1) Kişisel verileriniz, hizmet sunumu ve müşteri desteği amaçlarıyla işlenir.
(Score: 0.3962) 4) Veri saklama süresi, ilgili mevzuata uygun şekilde belirlenir
(Score: 0.3960) ; gerekli olmadığında veriler silinir veya anonimleştirilir.
(Score: 0.3630) Gizlilik politikamız aşağıdaki gibidir.
(Score: 0.3477) 2) Verileriniz, yasal yükümlülükler veya açık rızanız olmaksızın üçüncü taraflarla paylaşılmaz.
(Score: 0.3406) 3) Çerezler (cookies), deneyimi iyileştirmek ve analiz yapmak için kullanılabilir
(Score: 0.3131) . Tarayıcı ayarlarınızdan çerezleri yönetebilirsiniz

In [60]:
rerank_pairs = [[question, doc] for doc in candidates]
rerank_scores = reranker_model.compute_score(rerank_pairs)
rerank_scores = sigmoid(rerank_scores)
ranked_docs = sorted(zip(candidates, rerank_scores), key=lambda x: x[1], reverse=True)

print("After reranking:")
for i, (doc, score) in enumerate(ranked_docs, 1):
    print(f"{i}. (Score: {score:.4f}) {doc}")

After reranking:
1. (Score: 0.3900) 3) İade onayı sonrasında ücret iadesi 3-10 iş günü içinde gerçekleştirilir.
2. (Score: 0.0272) 1) Teslimat tarihinden itibaren 14 gün içinde iade talebi oluşturabilirsiniz. Ürün kullanılmamış olmalıdır.
3. (Score: 0.0208) İade politikamız aşağıdaki gibidir.
4. (Score: 0.0016) 2) Kargo takip numarası sipariş sayfasında yer alır. İade kargo bedeli kampanya dönemlerinde ücretsiz olabilir.
5. (Score: 0.0001) ; gerekli olmadığında veriler silinir veya anonimleştirilir.
6. (Score: 0.0000) 4) Veri saklama süresi, ilgili mevzuata uygun şekilde belirlenir
7. (Score: 0.0000) 1) Kişisel verileriniz, hizmet sunumu ve müşteri desteği amaçlarıyla işlenir.
8. (Score: 0.0000) 2) Verileriniz, yasal yükümlülükler veya açık rızanız olmaksızın üçüncü taraflarla paylaşılmaz.
9. (Score: 0.0000) Gizlilik politikamız aşağıdaki gibidir.
10. (Score: 0.0000) . Tarayıcı ayarlarınızdan çerezleri yönetebilirsiniz.
11. (Score: 0.0000) 3) Çerezler (cookies), deneyimi iyileştirmek v

# Değerlendirme

- Recall@k
- MRR
- nDCG

In [61]:
def recall_at_k(relevant: set, ranked_list: Sequence, k: int) -> float:
    if k <= 0:
        return 0.0
    hits = sum(1 for item in ranked_list[:k] if item in relevant)
    return hits / max(1, len(relevant))

def mean_reciprocal_rank(list_of_ranked: Iterable[Sequence], list_of_relevant: Iterable[set]) -> float:
    total, n = 0.0, 0
    for ranked, rel in zip(list_of_ranked, list_of_relevant):
        rr = 0.0
        for idx, item in enumerate(ranked, 1):
            if item in rel:
                rr = 1.0 / idx
                break
        total += rr
        n += 1
    return total / max(1, n)

def ndcg(ranked_list: Sequence, relevance: dict, k: int) -> float:
    def dcg(items):
        return sum((2 ** relevance.get(item, 0) - 1) / math.log2(i + 2) for i, item in enumerate(items[:k]))
    ideal = dcg(sorted(ranked_list, key=lambda x: relevance.get(x, 0), reverse=True))
    if ideal == 0:
        return 0.0
    return dcg(ranked_list) / ideal

In [67]:
relevant_chunks = set(chunk_index for chunk_index, chunk in enumerate(chunks) if chunk.metadata.get("source_id") == "policy_returns_tr")

print(f"Relevant chunk indices: {relevant_chunks}")
print(f"Total relevant chunks: {len(relevant_chunks)}")

# ranked_docs'tan sadece index'leri al
ranked_indices = []
for doc, score in ranked_docs:
    # Her doc'un chunks listesindeki index'ini bul
    for i, chunk in enumerate(chunks):
        if chunk.text == doc:
            ranked_indices.append(i)
            break

print(f"\nRanked indices: {ranked_indices}")

# Recall@k hesapla (k=3, k=5)
for k in [3, 5]:
    recall = recall_at_k(relevant_chunks, ranked_indices, k)
    print(f"Recall@{k}: {recall:.4f}")

# MRR hesapla
mrr = mean_reciprocal_rank([ranked_indices], [relevant_chunks])
print(f"\nMean Reciprocal Rank: {mrr:.4f}")

# nDCG hesapla
# Relevance skorlarını ayarla (relevant=2, irrelevant=0)
relevance_scores = {}
for i in range(len(chunks)):
    if i in relevant_chunks:
        relevance_scores[i] = 2
    else:
        relevance_scores[i] = 0

for k in [3, 5, len(ranked_indices)]:
    ndcg_score = ndcg(ranked_indices, relevance_scores, k)
    print(f"nDCG@{k}: {ndcg_score:.4f}")


Relevant chunk indices: {0, 1, 2, 3}
Total relevant chunks: 4

Ranked indices: [3, 1, 0, 2, 10, 9, 5, 6, 4, 8, 7]
Recall@3: 0.7500
Recall@5: 1.0000

Mean Reciprocal Rank: 1.0000
nDCG@3: 1.0000
nDCG@5: 1.0000
nDCG@11: 1.0000


# Hands on!

## Başka sorular için tüm hesapları yapıp, metrik yorumlayalım.